In [2]:
import os
from pathlib import Path
import pandas as pd


# 阿里妈妈数据集
download from https://tianchi.aliyun.com/dataset/147588

## 任务
本次比赛以阿里电商广告为研究对象，提供了淘宝平台的海量真实交易数据，参赛选手通过人工智能技术构建预测模型预估用户的购买意向，即给定广告点击相关的用户（user）、广告商品（ad）、检索词（query）、上下文内容（context）、商店（shop）等信息的条件下预测广告产生购买行为的概率（pCVR），形式化定义为：pCVR=P(conversion=1 | query, user, ad, context, shop)。

结合淘宝平台的业务场景和不同的流量特点，我们定义了以下两类挑战：

（1）日常的转化率预估

（2）特殊日期的转化率预估

## 数据说明

本次比赛为参赛选手提供了5类数据（基础数据、广告商品信息、用户信息、上下文信息和店铺信息）。基础数据表提供了搜索广告最基本的信息，以及“是否交易”的标记。广告商品信息、用户信息、上下文信息和店铺信息等4类数据，提供了对转化率预估可能有帮助的辅助信息。

### 基础数据

| 字段        | 解释                                                         |
| :---------- | :----------------------------------------------------------- |
| instance_id | 样本编号，Long                                               |
| is_trade    | 是否交易的标记位，Int类型；取值是0或者1，其中1 表示这条样本最终产生交易，0 表示没有交易 |
| item_id     | 广告商品编号，Long类型                                       |
| user_id     | 用户的编号，Long类型                                         |
| context_id  | 上下文信息的编号，Long类型                                   |
| shop_id     | 店铺的编号，Long类型                                         |

### 广告商品信息

| 字段                 | 解释                                                         |
| :------------------- | :----------------------------------------------------------- |
| item_id              | 广告商品编号，Long类型                                       |
| item_category_list   | 广告商品的的类目列表，String类型；从根类目（最粗略的一级类目）向叶子类目（最精细的类目）依次排列，数据拼接格式为 "category_0;category_1;category_2"，其中 category_1 是 category_0 的子类目，category_2 是 category_1 的子类目 |
| item_property_list   | 广告商品的属性列表，String类型；数据拼接格式为 "property_0;property_1;property_2"，各个属性没有从属关系 |
| item_brand_id        | 广告商品的品牌编号，Long类型                                 |
| item_city_id         | 广告商品的城市编号，Long类型                                 |
| item_price_level     | 广告商品的价格等级，Int类型；取值从0开始，数值越大表示价格越高 |
| item_sales_level     | 广告商品的销量等级，Int类型；取值从0开始，数值越大表示销量越大 |
| item_collected_level | 广告商品被收藏次数的等级，Int类型；取值从0开始，数值越大表示被收藏次数越大 |
| item_pv_level        | 广告商品被展示次数的等级，Int类型；取值从0开始，数值越大表示被展示次数越大 |

### 用户信息

| 字段               | 解释                                                         |
| :----------------- | :----------------------------------------------------------- |
| user_id            | 用户的编号，Long类型                                         |
| user_gender_id     | 用户的预测性别编号，Int类型；0表示女性用户，1表示男性用户，2表示家庭用户 |
| user_age_level     | 用户的预测年龄等级，Int类型；数值越大表示年龄越大            |
| user_occupation_id | 用户的预测职业编号，Int类型                                  |
| user_star_level    | 用户的星级编号，Int类型；数值越大表示用户的星级越高          |

### 上下文信息

| 字段                      | 解释                                                         |
| :------------------------ | :----------------------------------------------------------- |
| context_id                | 上下文信息的编号，Long类型                                   |
| context_timestamp         | 广告商品的展示时间，Long类型；取值是以秒为单位的Unix时间戳，以1天为单位对时间戳进行了偏移 |
| context_page_id           | 广告商品的展示页面编号，Int类型；取值从1开始，依次增加；在一次搜索的展示结果中第一屏的编号为1，第二屏的编号为2 |
| predict_category_property | 根据查询词预测的类目属性列表，String类型；数据拼接格式为“category_A:property_A_1,property_A_2,property_A_3;category_B:-1;category_C:property_C_1,property_C_2” ，其中 category_A、category_B、category_C 是预测的三个类目；property_B 取值为-1，表示预测的第二个类目 category_B 没有对应的预测属性 |

### 店铺信息

| 字段                      | 解释                                                         |
| :------------------------ | :----------------------------------------------------------- |
| shop_id                   | 店铺的编号，Long类型                                         |
| shop_review_num_level     | 店铺的评价数量等级，Int类型；取值从0开始，数值越大表示评价数量越多 |
| shop_review_positive_rate | 店铺的好评率，Double类型；取值在0到1之间，数值越大表示好评率越高 |
| shop_star_level           | 店铺的星级编号，Int类型；取值从0开始，数值越大表示店铺的星级越高 |
| shop_score_service        | 店铺的服务态度评分，Double类型；取值在0到1之间，数值越大表示评分越高 |
| shop_score_delivery       | 店铺的物流服务评分，Double类型；取值在0到1之间，数值越大表示评分越高 |
| shop_score_description    | 店铺的描述相符评分，Double类型；取值在0到1之间，数值越大表示评分越高 |

在上述各张数据表中，绝大部分样本包含了完整的字段数据，也有少部分样本缺乏特定字段的数据。如果一条样本的某个字段为“-1”，表示这个样本的对应字段缺乏数据。


In [17]:
current_path = Path.cwd()
print(f"current path : {current_path}")
data_dir = current_path.parent.parent / Path(r"data\ijcai_18_alimama\round1_ijcai_18_train\round1_ijcai_18_train_20180301.txt")
print(f"data dir:{data_dir}")
chunk_size = 10000

current path : c:\Users\zhangyu\git\RecJourney\models\MMOE
data dir:c:\Users\zhangyu\git\RecJourney\data\ijcai_18_alimama\round1_ijcai_18_train\round1_ijcai_18_train_20180301.txt


In [19]:
label_column = "is_trade"
id_columns = [
    "instance_id",
    "item_id",
    "user_id",
    "context_id",
    "shop_id"
]

item_columns = [
    "item_category_list",
    "item_property_list",
    "item_brand_id",
    "item_city_id",
    "item_price_level",
    "item_sales_level",
    "item_collected_level",
    "item_pv_level",
]


user_columns = [
    "user_gender_id",
    "user_age_level",
    "user_occupation_id",
    "user_star_level"
]

context_columns = [
    "context_timestamp",
    "context_page_id",
    "predict_category_property",
]

shop_columns = [
    "shop_review_num_level",
    "shop_review_positive_rate",
    "shop_star_level",
    "shop_score_service",
    "shop_score_delivery",
    "shop_score_description"
]


In [14]:
df = pd.read_csv(data_dir, sep=' ', chunksize=chunk_size)
chunk = df.get_chunk(chunk_size)
print(chunk.columns)
chunk


Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'is_trade'],
      dtype='object')


,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.000000,1.000000,1.000000,0
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.000000,1.000000,1.000000,0
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.000000,1.000000,1.000000,0
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4016,509660095530134768:-1;5799347067982556520:-1;7...,6765930309048922341,4,1.0,5002,1.000000,1.000000,1.000000,0
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.000000,1.000000,1.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5914623189257523594,1502399167133432846,7908382889764677758;8277336076276184272,5977512434884267894;2072967855524022579;513128...,7024713306861377790,7534238860363577544,7,11,11,17,...,4003,"8277336076276184272:2636395404473730413,820214...",4954894036019286224,17,1.0,5015,0.973817,0.971494,0.985987,1
9996,5082808267147521726,2723541945574109807,7908382889764677758;509660095530134768,2072967855524022579;5131280576272319091;263639...,7024713306861377790,7534238860363577544,7,15,16,20,...,4001,509660095530134768:-1;2011981573061447208:-1;5...,4954894036019286224,17,1.0,5015,0.973817,0.971494,0.985987,0
9997,5016137884844600866,2723541945574109807,7908382889764677758;509660095530134768,2072967855524022579;5131280576272319091;263639...,7024713306861377790,7534238860363577544,7,15,16,20,...,4001,"509660095530134768:9148482949976129397,2636395...",4954894036019286224,17,1.0,5015,0.973817,0.971494,0.985987,0
9998,6811589004997751056,4399478434881285132,7908382889764677758;5755694407684602296,2072967855524022579;5131280576272319091;263639...,7024713306861377790,7534238860363577544,6,12,13,17,...,4014,5755694407684602296:2636395404473730413;201198...,4954894036019286224,17,1.0,5015,0.973817,0.971494,0.985987,0


In [22]:
chunk.describe()

,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,...,context_timestamp,context_page_id,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade
count,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,...,1.000000e+04,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4.604355e+18,4.116029e+18,4.881201e+18,4.832578e+18,6.678900,12.923500,13.772900,18.658000,4.621757e+18,0.193100,...,1.537256e+09,4002.923500,5.580179e+18,18.204300,0.996949,5015.316700,0.972273,0.972043,0.977948,0.022700
std,2.671942e+18,2.677171e+18,2.648235e+18,2.390162e+18,1.073457,2.455209,2.272409,1.839014,2.650934e+18,0.496827,...,1.951456e+04,3.466937,2.767175e+18,2.531522,0.007038,2.341356,0.009057,0.008916,0.010103,0.148953
min,1.000728e+14,3.612887e+16,1.017980e+16,1.962573e+17,2.000000,1.000000,1.000000,5.000000,8.517544e+14,-1.000000,...,1.537200e+09,4001.000000,1.538377e+17,4.000000,0.944444,5002.000000,0.934025,0.932637,0.920000,0.000000
25%,2.285392e+18,2.079092e+18,2.220007e+18,3.122722e+18,6.000000,11.000000,13.000000,18.000000,2.324192e+18,0.000000,...,1.537242e+09,4001.000000,2.025577e+18,17.000000,0.997040,5014.000000,0.967210,0.966481,0.975442,0.000000
50%,4.587383e+18,3.369992e+18,5.818353e+18,4.644954e+18,7.000000,14.000000,14.000000,19.000000,4.647357e+18,0.000000,...,1.537258e+09,4001.000000,6.597981e+18,18.000000,1.000000,5015.000000,0.976664,0.976344,0.978433,0.000000
75%,6.955847e+18,5.720824e+18,7.066303e+18,7.322157e+18,7.000000,15.000000,15.000000,20.000000,6.892886e+18,0.000000,...,1.537273e+09,4003.000000,8.242680e+18,21.000000,1.000000,5018.000000,0.978163,0.977956,0.983678,0.000000
max,9.223041e+18,9.199875e+18,9.057103e+18,9.070708e+18,9.000000,16.000000,17.000000,21.000000,9.220592e+18,2.000000,...,1.537286e+09,4020.000000,9.121505e+18,22.000000,1.000000,5018.000000,1.000000,1.000000,1.000000,1.000000
